# Data Crawling
For the training of the two sub-networks of the two networks we need data. In this project the selection of training data is an important creative decision. Since Machine Learning algorithms try to minimize some kind of error, it is crutial how this error is defined. The networks will try to produce sound as close as possible to their training data. Thus the produced sounds will be very different, for different training data. As stated above, we want to give as many creative decision as possible to machines, since the goal is a meta-human sound. Therefor we've decided to use yet another machine learning algorithm to select the training data: YouTube's recomendation algorithm.

The following code controls the mouse of the users system to collect YouTube links. For this the user has to open a YouTube video on a Full-HD screen. The code will automatically click on the next recomendation and copy the URL. That can be used to download the sound of the YouTube video, e.g. via the program MediaHuman.

WARNING: This code controls the mouse! You can only stop it by pressing and holding down the space bar!

In [1]:
import mouse, keyboard
import time

def click_next_video():
    mouse.move(504, 700)
    time.sleep(5)
    mouse.click()
    time.sleep(2)
    keyboard.send("shift+n")
    time.sleep(1)

def copy_link():
    mouse.move(504, 50)
    mouse.click()
    time.sleep(0.6)
    keyboard.send("ctrl+c")

if __name__ == '__main__':

    # while True:
    #    print(mouse.get_position())
    time.sleep(5) # time to change the window to Youtube

    for i in range(1000):
        if keyboard.is_pressed("space"):
            exit()
        copy_link()
        click_next_video()
        print(i)



ModuleNotFoundError: No module named 'mouse'

During testing of this project, we've recognized that songs with full instrumentation are too complex. Rather we want to use single instruments or vocals. Again, we don't need a human to decide which signal counts as "vocals". We can use another machine learning algorithm to extract stems from the downloaded song. We used spleeter (more info here: https://github.com/deezer/spleeter)


In [ ]:
import time
import librosa
import soundfile as sf
from spleeter.separator import Separator
from spleeter.audio.adapter import AudioAdapter
import glob
import numpy as np


def _check_rms(data, path="Unkown"):
    rms = librosa.feature.rms(y=data, frame_length=data.shape[0])
    if rms[0][0] > 0.00000001:
        print("RMS of song " + str(path) + " is: " + str(rms[0][0]))
        return 0
    else:
        print("I don't think there were vocals in the song: " + str(path))
        return 1  # The RMS Level is very low -> probably the song didn't have vocals


if __name__ == '__main__':
    separator = Separator('spleeter:2stems')
    subfolder = 1500
    list_of_files = glob.glob('demo_data\\YouTubeSongs\\' + str(subfolder) + '\\*')
   # print(list_of_files)
    start_time = time.time()

    audio_loader = AudioAdapter.default()
    sample_rate = 44100

    # Perform the Separation
    for i,file in enumerate(list_of_files):
        waveform, _ = audio_loader.load(file, sample_rate=sample_rate)
        prediction = separator.separate(waveform)
        vocals = prediction["vocals"]
        empty = _check_rms(vocals, file)
        save_path = "demo_data\\stems\\voclas" + str(i + (subfolder-100)) +".wav"
        if empty<1:
            sf.write(file=save_path, data=vocals, samplerate=44100)
        print(str(i+1) + " of " + str(len(list_of_files)) +" Songs done")
    end_time = time.time()

    print("took me " + str(end_time-start_time) + " seconds to finish " + str(len(list_of_files)) + " songs")